In [1]:
import numpy as np; import pandas as pd

In [2]:
# kolom dalam tabel dapat mengandung nilai berbeda yang berulang, untuk melihat distinct values dapat dengan unique dan value_counts

values = pd.Series(['apple', 'orange', 'apple', 'apple'] * 2)

In [3]:
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [7]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [8]:
pd.unique(values.index)

array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int64)

In [9]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

In [10]:
# dimension tables is best practice for storing repeated values
# containing distinct values and primary observation as integer keys referencing the dimension table
# This representation as integers is called 'categorical' / dict-encoded representation

values = pd.Series([0, 1, 0, 0] * 2)

In [11]:
dim = pd.Series(['apple', 'orange'])

In [12]:
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [13]:
dim

0     apple
1    orange
dtype: object

In [14]:
dim.take(values)

# take method to restore original Series of strings

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

In [17]:
# Categorical Type in pandas

fruits = ['apple', 'orange', 'apple', 'apple'] * 2

In [18]:
N = len(fruits)

In [22]:
df = pd.DataFrame({'fruit': fruits,
                  'basket_id': np.arange(N),
                  'count': np.random.randint(3, 15, size=N),
                  'weight': np.random.uniform(0, 4, size=N)}, 
                 columns=['basket_id', 'fruit', 'count', 'weight'])

In [23]:
df

,basket_id,fruit,count,weight
0,0,apple,3,3.508139
1,1,orange,5,2.696660
2,2,apple,3,3.028682
3,3,apple,14,1.402291
4,4,apple,3,3.023449
5,5,orange,6,3.084172
6,6,apple,5,1.116896
7,7,apple,6,2.751526


In [24]:
fruit_cat = df['fruit'].astype('category')

In [25]:
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [26]:
c = fruit_cat.values

In [28]:
type(c)

# values for fruit_cat are not NumPy array, but an instance of pandas

pandas.core.arrays.categorical.Categorical

In [29]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [30]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [31]:
df['fruit'] = df['fruit'].astype('category')

# convert DataFrame column to categorical by assigning the converted result

In [32]:
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [33]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])

In [34]:
my_categories

[foo, bar, baz, foo, bar]
Categories (3, object): [bar, baz, foo]

In [35]:
categories = ['foo', 'bar', 'baz']

In [36]:
codes = [0, 1, 2, 0, 0, 1]

# from_codes capat digunakan jika kode kategorinya telah dipunyai

In [37]:
my_cats_2 = pd.Categorical.from_codes(codes, categories)

In [38]:
my_cats_2

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

In [39]:
ordered_cat = pd.Categorical.from_codes(codes, categories, ordered=True)

# categorical conversion does NOT assume specific categories ordering

In [40]:
ordered_cat

# [foo < bar < baz] this means foo is 1st, bar is 2nd, baz is 3rd

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

In [41]:
my_cats_2.as_ordered()

# this will ordering the categorical instance

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

In [42]:
# Computations with Categoricals

np.random.seed(12345)

In [43]:
draws = np.random.randn(1000)

In [44]:
draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

In [45]:
bins = pd.qcut(draws, 4)

# compute quartile binning of this data

# ini APAAA?

In [46]:
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

In [47]:
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])

In [48]:
bins

[Q2, Q3, Q2, Q2, Q4, ..., Q3, Q2, Q1, Q3, Q4]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [49]:
bins.codes[:10]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

In [50]:
bins = pd.Series(bins, name='quartile')

In [51]:
results = (pd.Series(draws)
           .groupby(bins)
           .agg(['count', 'min', 'max'])
          .reset_index())

# this will categorize draws according to bins value (which is the categorical version of draws)
# groupby will group them
# agg will compute the functions

In [52]:
results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


In [53]:
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [54]:
# Better performance with categoricals

# converting to categorical can yield substantial overall performance gains. 
# A categorical version of a DataFrame column will often use significantly less memory, too. 

N = 10000000

In [55]:
draws = pd.Series(np.random.randn(N))

In [56]:
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))

In [57]:
categories = labels.astype('category')

In [58]:
labels.memory_usage()

80000128

In [59]:
categories.memory_usage()

10000320

In [60]:
%time _ = labels.astype('category')

Wall time: 1.15 s


In [61]:
# Categorical Methods

# special methods for categorical data

s = pd.Series(['a', 'b', 'c', 'd'] * 2)

In [62]:
cat_s = s.astype('category')

In [63]:
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

In [64]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [65]:
cat_s.cat.categories

# special attribute cat provides access to categorical methods

Index(['a', 'b', 'c', 'd'], dtype='object')

In [66]:
actual_categories = ['a', 'b', 'c', 'd', 'e']

In [67]:
cat_s2 = cat_s.cat.set_categories(actual_categories)

# this set_categories will change categories of preceding categorical data

In [68]:
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]

In [69]:
cat_s.value_counts()

d    2
c    2
b    2
a    2
dtype: int64

In [70]:
cat_s2.value_counts()

# new categories will be reflected in operations that use them

d    2
c    2
b    2
a    2
e    0
dtype: int64

In [71]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]

# categorical are often used as a convenient tool for memory savings and better performance.

In [72]:
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): [a, b, c, d]

In [73]:
cat_s3.cat.remove_unused_categories()

# remove_unused_categories method to trim unobserved categories

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): [a, b]

In [75]:
# creating dummy variables for modelling for statistics and machine learning
# bikin data biner dari kategori gitu buat dihitung

cat_s = pd.Series(['a', 'b', 'c', 'd'], dtype='category')

In [76]:
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1


In [77]:
# ADVANCED groupby USE

# Group Transforms and 'Unwrapped' GroupBys

# transform method - similar to apply but with more constrain on the kind of function you can use

df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                  'value': np.arange(12.)})

In [78]:
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [79]:
g = df.groupby('key').value

In [81]:
h = df.groupby('key')

In [80]:
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [82]:
h.mean()

,value
key,
a,4.5
b,5.5
c,6.5


In [83]:
g.transform(lambda x: x.mean())

# for x, replace with x.mean()

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [84]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [85]:
g.transform(lambda x: x * 2)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [86]:
g.transform(lambda x: x.rank(ascending=False))

# ranks in descending order

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [87]:
def normalize(x):
    return(x - x.mean()) / x.std()

In [88]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [89]:
g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [90]:
# mean or sum is faster than general apply function

g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [91]:
normalized = (df['value'] - g.transform('mean')) / g.transform('std')

In [92]:
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [93]:
# Grouped Time Resampling

#  resample method is semantically a group operation based on a time intervalization.

N = 15

In [94]:
times = pd.date_range('2017-05-20 00:00', freq='1min', periods=N)

In [95]:
df = pd.DataFrame({'time': times,
                  'value': np.arange(N)})

In [96]:
df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


In [97]:
df.set_index('time').resample('5min').count()

# index by time

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


In [99]:
df2 = pd.DataFrame({'time': times.repeat(3),
                   'key': np.tile(['a', 'b', 'c'], N),
                   'value': np.arange(N * 3.)})

In [100]:
df2[:7]

,time,key,value
0,2017-05-20 00:00:00,a,0.0
1,2017-05-20 00:00:00,b,1.0
2,2017-05-20 00:00:00,c,2.0
3,2017-05-20 00:01:00,a,3.0
4,2017-05-20 00:01:00,b,4.0
5,2017-05-20 00:01:00,c,5.0
6,2017-05-20 00:02:00,a,6.0


In [101]:
# resampling for each value of 'key' (time) -> pd.TimeGrouper
# time must be the index of the Series/DF

time_key = pd.TimeGrouper('5min')

AttributeError: module 'pandas' has no attribute 'TimeGrouper'

In [ ]:
resampled = (df2.set_index('time')
            .groupby(['key', time_key]).sum())

In [ ]:
resampled

In [ ]:
resampled.reset_index

In [ ]:
# TECHNIQUES FOR METHOD CHAINING

# assign method
# pipe method